# OrbitalElements interactive examples

## Working environment & Packages

Set the working environment to the repository root.

In [ ]:
using Plots;
plotlyjs();

In [ ]:
using Pkg
Pkg.activate(@__DIR__()*"/../.")
import OrbitalElements as OE

# Generic parameters

## Potential

In [ ]:
G, M, bc = 1.0, 1.0, 1.0
ψ(r::Float64)   = OE.ψIsochrone(r,bc,M,G)
dψ(r::Float64)  = OE.dψIsochrone(r,bc,M,G)
d2ψ(r::Float64) = OE.d2ψIsochrone(r,bc,M,G)
Ω₀ = OE.Ω₀Isochrone(bc,M,G);

## Parameters

In [ ]:
# Radii for frequency truncations
rmin, rmax = 0.1, 100.0
# Tolerances / Taylor expansions
EDGE = 0.01
TOLECC = 0.01
TOLA = 1.0
NINT = 32

params = OE.OrbitalParameters(Ω₀=Ω₀,rmin=rmin,rmax=rmax,
                                            EDGE=EDGE,TOLA=TOLA,TOLECC=TOLECC,
                                            NINT=NINT);

# Frequency computation

Frequency values along a constant eccentricity line. Example using the isochrone potential for which analytical expression of the frequencies are known. Comparing analytical and numerical results.

In [ ]:
function FrequenciesE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (2,length(tabe))
    for j=1:ne
        Ω1, Ω2 = OE.ComputeFrequenciesAE(ψ,dψ,d2ψ,a,tabe[j],params)
        @inbounds results[1,j], results[2,j] = Ω1, Ω2
    end
end;
function IsochroneFrequenciesE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (2,length(tabe))
    for j=1:ne
        Ω1true, Ω2true = OE.IsochroneOmega12FromAE(a,tabe[j])
        @inbounds results[1,j], results[2,j] = Ω1true, Ω2true
    end
end;

In [ ]:
ne = 200;
tabe = collect(LinRange(0.,1.,ne));
tabΩ = zeros(2,ne);
tabΩIso = zeros(2,ne);

In [ ]:
a = 1.0

# Compute
FrequenciesE!(a,tabe,tabΩ)
IsochroneFrequenciesE!(a,tabe,tabΩIso)

# Plot
xlabelΩ = ["e" "e"]
ylabelΩ = ["Ω1" "Ω2"]
p1 = plot(tabe,tabΩ[1,:],xlabel=xlabelΩ[1],ylabel=ylabelΩ[1],label="Numerical");
plot!(tabe,tabΩIso[1,:],label="True")
p2 = plot(tabe,tabΩ[2,:],xlabel=xlabelΩ[2],ylabel=ylabelΩ[2],label=false);
plot!(tabe,tabΩIso[2,:],label=false)
plot(p1,p2,layout=(1,2),size=(900,300))

# Frequency derivatives computation

Frequency values along a constant eccentricity line. Example using the isochrone potential for which analytical expression of the frequencies are known. Comparing analytical and numerical results.

In [ ]:
function FrequenciesDerivE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (6,length(tabe))
    for j=1:ne
        Ω1, Ω2, ∂Ω1∂a, ∂Ω2∂a, ∂Ω1∂e, ∂Ω2∂e = OE.ComputeFrequenciesAEWithDeriv(ψ,dψ,d2ψ,a,tabe[j],params)
        @inbounds results[1,j], results[2,j], results[3,j], results[4,j], results[5,j], results[6,j]= Ω1, Ω2, ∂Ω1∂a, ∂Ω2∂a, ∂Ω1∂e, ∂Ω2∂e
    end
end;

In [ ]:
ne = 200;
tabe = collect(LinRange(0.,1.,ne));
tabderΩ = zeros(6,ne);

In [ ]:
a = 1.0

# Compute
FrequenciesDerivE!(a,tabe,tabderΩ)

# Plot
xlabelderΩ = ["" "" "" "" "e" "e"]
ylabelderΩ = ["Ω1" "Ω2" "∂Ω1∂a" "∂Ω2∂a" "∂Ω1∂e" "∂Ω2∂e"]
plot(tabe,transpose(tabderΩ),layout=(3,2),xlabel=xlabelderΩ,ylabel=ylabelderΩ,legend=false,size=(900,600))

# Energy, angular momentum and derivatives computation

In [ ]:
function ELDerivE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (6,length(tabe))
    for j=1:ne
        E, L, ∂E∂a, ∂L∂a, ∂E∂e, ∂L∂e = OE.ComputeELAEWithDeriv(ψ,dψ,a,tabe[j],params)
        @inbounds results[1,j], results[2,j], results[3,j], results[4,j], results[5,j], results[6,j]= E, L, ∂E∂a, ∂L∂a, ∂E∂e, ∂L∂e
    end
end;

In [ ]:
ne = 200;
tabe = collect(LinRange(0.,1.,ne));
tabderEL = zeros(6,ne);

In [ ]:
a = 1.0

# Compute
ELDerivE!(a,tabe,tabderEL)

# Plot
xlabelEL = ["" "" "" "" "e" "e"]
ylabelEL = ["E" "L" "∂E∂a" "∂L∂a" "∂E∂e" "∂L∂e"]
plot(tabe,transpose(tabderEL),layout=(3,2),xlabel=xlabelEL,ylabel=ylabelEL,legend=false,size=(900,600))

# Inversions

In [ ]:
na, ne = 100, 101;
amin, amax = 0.01, 10.;
taba = collect(LinRange(amin,amax,na));
tabe = collect(LinRange(0.,1.,ne));

tabAE = zeros(Float64,2,na*ne);
restabAE = zeros(Float64,2,na*ne);
errtabAE = zeros(Float64,na,ne);

## (a,e) &harr; (E,L) 

In [ ]:
tabEL = zeros(Float64,2,na*ne);

In [ ]:
function AEToELmapping(taba::Vector{Float64},tabe::Vector{Float64},tabAE::Matrix{Float64},
                       restabAE::Matrix{Float64},tabEL::Matrix{Float64},taberr::Matrix{Float64})
    
    na, ne = length(taba), length(tabe)
    
    count = 1
    for ka = 1:na
        for ke = 1:ne
            a, e = taba[ka], tabe[ke]
            
            E, L = OE.ELFromAE(ψ,dψ,a,e,params)
            aback, eback = OE.ComputeAEFromEL(ψ,dψ,E,L,params)
            
            tabAE[1,count], tabAE[2,count] = a, e 
            tabEL[1,count], tabEL[2,count] = E, L
            restabAE[1,count], restabAE[2,count] = aback, eback
            taberr[ka,ke] = abs((aback-a)/a) + abs(eback-e)
            
            count += 1
        end
    end
end;

In [ ]:
AEToELmapping(taba,tabe,tabAE,restabAE,tabEL,errtabAE)

In [ ]:
# (E,L) space shape
scatter(tabEL[2,:],tabEL[1,:],markersize=0.01)

In [ ]:
# (a,e)->(E,L)->(a,e) result
scatter(restabAE[1,:],restabAE[2,:],markersize=0.01)

In [ ]:
# Error made on (a,e)->(E,L)->(a,e) in log scale
# Trouble for radial orbits and close to the center
heatmap(taba,tabe,transpose(log10.(errtabAE)))

## (a,e) &harr; (Jr,L) 

In [ ]:
tabJL = zeros(Float64,2,na*ne);

In [ ]:
function AEToActionsmapping(taba::Vector{Float64},tabe::Vector{Float64},tabAE::Matrix{Float64},
                       restabAE::Matrix{Float64},tabJL::Matrix{Float64},taberr::Matrix{Float64})
    
    na, ne = length(taba), length(tabe)
    
    count = 1
    for ka = 1:na
        for ke = 1:ne
            a, e = taba[ka], tabe[ke]
            
            J, L = OE.ComputeActionsAE(ψ,dψ,a,e,params)
            aback, eback = OE.ComputeAEFromActions(ψ,dψ,J,L,params)
            
            tabAE[1,count], tabAE[2,count] = a, e 
            tabJL[1,count], tabJL[2,count] = J, L
            restabAE[1,count], restabAE[2,count] = aback, eback
            taberr[ka,ke] = abs((aback-a)/a) + abs(eback-e)
            
            count += 1
        end
    end
end;

In [ ]:
AEToActionsmapping(taba,tabe,tabAE,restabAE,tabJL,errtabAE)

In [ ]:
# J,L space shape
scatter(tabJL[2,:],tabJL[1,:],markersize=0.01)

In [ ]:
# (a,e)->(J,L)->(a,e) result
scatter(restabAE[1,:],restabAE[2,:],markersize=0.01)

In [ ]:
# Error made on (a,e)->(J,L)->(a,e) in log scale
# Trouble for radial orbits and close to the center
heatmap(taba,tabe,transpose(log10.(errtabAE)))

## (a,e) &harr; ($\Omega$1,$\Omega$2) 

In [ ]:
tabΩ1Ω2 = zeros(Float64,2,na*ne);

In [ ]:
function AEToFrequenciesmapping(taba::Vector{Float64},tabe::Vector{Float64},tabAE::Matrix{Float64},
                       restabAE::Matrix{Float64},tabΩ1Ω2::Matrix{Float64},taberr::Matrix{Float64})
    
    na, ne = length(taba), length(tabe)
    
    count = 1
    for ka = 1:na
        for ke = 1:ne
            a, e = taba[ka], tabe[ke]
            
            Ω1, Ω2 = OE.ComputeFrequenciesAE(ψ,dψ,d2ψ,a,e,params)
            aback, eback = OE.ComputeAEFromFrequencies(ψ,dψ,d2ψ,Ω1,Ω2,params)
            
            tabAE[1,count], tabAE[2,count] = a, e 
            tabΩ1Ω2[1,count], tabΩ1Ω2[2,count] = Ω1, Ω2
            restabAE[1,count], restabAE[2,count] = aback, eback
            taberr[ka,ke] = abs((aback-a)/a) + abs(eback-e)
            
            count += 1
        end
    end
end;

In [ ]:
AEToFrequenciesmapping(taba,tabe,tabAE,restabAE,tabΩ1Ω2,errtabAE)

In [ ]:
# Frequencies space shape
scatter(tabΩ1Ω2[1,:],tabΩ1Ω2[2,:],markersize=0.01)

In [ ]:
# (a,e)->(Ω1,Ω2)->(a,e) result
scatter(restabAE[1,:],restabAE[2,:],markersize=0.01)

In [ ]:
# Error made on (a,e)->(Ω1,Ω2)->(a,e) in log scale
# Trouble for radial orbits and close to the center
heatmap(taba,tabe,transpose(log10.(errtabAE)))

# Resonance variables